In [ ]:
!pip install catboost

In [ ]:
### Sagar Raut PRN : 240340128027
### Ameya Panshikar PRN : 240340128002

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Load the training and testing datasets
tab_train = pd.read_csv(r"/content/train.csv")
tab_test = pd.read_csv(r"/content/test.csv")

In [ ]:
print(tab_train.info())
print(tab_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848835 entries, 0 to 848834
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   row_id      848835 non-null  int64 
 1   time        848835 non-null  object
 2   x           848835 non-null  int64 
 3   y           848835 non-null  int64 
 4   direction   848835 non-null  object
 5   congestion  848835 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 38.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_id     2340 non-null   int64 
 1   time       2340 non-null   object
 2   x          2340 non-null   int64 
 3   y          2340 non-null   int64 
 4   direction  2340 non-null   object
dtypes: int64(3), object(2)
memory usage: 91.5+ KB
None


In [ ]:
le = LabelEncoder()
tab_train['direction'] = le.fit_transform(tab_train['direction'])
tab_test['direction'] = le.transform(tab_test['direction'])

In [ ]:
X_train = tab_train.iloc[:,1:-1]
y_train = np.log1p(tab_train.iloc[:,-1])
X_test = tab_test.iloc[:,1:]

In [ ]:
X_train['time'] = X_train['time'].astype('datetime64[ns]')

In [ ]:
X_train['year'] = X_train['time'].dt.year
X_train['month'] = X_train['time'].dt.month
X_train['day'] = X_train['time'].dt.day
X_train['hour'] = X_train['time'].dt.hour
X_train['weekday'] = X_train['time'].dt.weekday

In [ ]:
X_test['time'] = X_test['time'].astype('datetime64[ns]')

In [ ]:
X_test['year'] = X_test['time'].dt.year
X_test['month'] = X_test['time'].dt.month
X_test['day'] = X_test['time'].dt.day
X_test['hour'] = X_test['time'].dt.hour
X_test['weekday'] = X_test['time'].dt.weekday

In [ ]:
X_train.drop('time',axis=1, inplace=True)

In [ ]:
X_test.drop('time',axis=1, inplace=True)

In [ ]:
lr = LinearRegression()
el = ElasticNet()
x_gbm = XGBRegressor()
l_gbm = LGBMRegressor()
c_gbm = CatBoostRegressor()

In [ ]:
# Train the models
lr.fit(X_train, y_train)
el.fit(X_train, y_train)
x_gbm.fit(X_train, y_train)
l_gbm.fit(X_train, y_train)
c_gbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040494 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 848835, number of used features: 7
[LightGBM] [Info] Start training from score 3.815882
Learning rate set to 0.118832
0:	learn: 0.4023023	total: 296ms	remaining: 4m 56s
1:	learn: 0.3899439	total: 568ms	remaining: 4m 43s
2:	learn: 0.3800916	total: 814ms	remaining: 4m 30s
3:	learn: 0.3714956	total: 1.03s	remaining: 4m 16s
4:	learn: 0.3650286	total: 1.3s	remaining: 4m 18s
5:	learn: 0.3584518	total: 1.56s	remaining: 4m 19s
6:	learn: 0.3522119	total: 1.81s	remaining: 4m 17s
7:	learn: 0.3475348	total: 2.1s	remaining: 4m 20s
8:	learn: 0.3426041	total: 2.4s	remaining: 4m 23s
9:	learn: 0.3398792	total: 2.57s	remaining: 4m 14s
10:	learn: 0.3362334	total: 2.77s	remaining: 4m 9s
11:	lear

In [ ]:
X_test.head()

x  y  direction  year  month  day  hour  weekday
0  0  0          0  1991      9   30    12        0
1  0  0          1  1991      9   30    12        0
2  0  0          4  1991      9   30    12        0
3  0  1          0  1991      9   30    12        0
4  0  1          1  1991      9   30    12        0

In [ ]:
# Make predictions on the testing set
y_pred_lr = lr.predict(X_test)
y_pred_el = el.predict(X_test)
y_pred_lgb = l_gbm.predict(X_test)
y_pred_xgb = x_gbm.predict(X_test)
y_pred_cgb = c_gbm.predict(X_test)

In [ ]:
# Make predictions on the testing set
y_pred_lr = np.expm1(y_pred_lr)
y_pred_el = np.expm1(y_pred_el)
y_pred_lgb = np.expm1(y_pred_lgb)
y_pred_xgb = np.expm1(y_pred_xgb)
y_pred_cgb = np.expm1(y_pred_cgb)

In [ ]:
# Create submission files for each model
submit_lr = pd.DataFrame({'row_id': tab_test['row_id'], 'congestion': y_pred_lr})
submit_lr.to_csv('sbt_tabular_lr.csv', index=False)

In [ ]:
# Create submission files for each model
submit_el = pd.DataFrame({'row_id': tab_test['row_id'], 'congestion': y_pred_el})
submit_el.to_csv('sbt_tabular_el.csv', index=False)

In [ ]:
# Create submission files for each model
submit_lgb = pd.DataFrame({'row_id': tab_test['row_id'], 'congestion': y_pred_lgb})
submit_lgb.to_csv('sbt_tabular_lgb.csv', index=False)

In [ ]:
# Create submission files for each model
submit_xgb = pd.DataFrame({'row_id': tab_test['row_id'], 'congestion': y_pred_xgb})
submit_xgb.to_csv('sbt_tabular_xgb.csv', index=False)

In [ ]:
# Create submission files for each model
submit_cgb = pd.DataFrame({'row_id': tab_test['row_id'], 'congestion': y_pred_cgb})
submit_cgb.to_csv('sbt_tabular_cgb.csv', index=False)